# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [8]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

d:\Users\hatem.kamal\Desktop\DE udacity\Test\CassandraPractice\project-1b-project-templateipynb


In [9]:
file_path_list

['d:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\project-1b-project-templateipynb/data\\2018-11-01-events.csv',
 'd:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\project-1b-project-templateipynb/data\\2018-11-02-events.csv',
 'd:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\project-1b-project-templateipynb/data\\2018-11-03-events.csv',
 'd:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\project-1b-project-templateipynb/data\\2018-11-04-events.csv',
 'd:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\project-1b-project-templateipynb/data\\2018-11-05-events.csv',
 'd:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\project-1b-project-templateipynb/data\\2018-11-06-events.csv',
 'd:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\project-1b-project-templateipynb/data\\2018-11-07-events.csv',
 'd:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\Ca

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [11]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [13]:
import pandas as pd

import os
from dotenv import load_dotenv
# import pathlib

from os.path import join, dirname

path = 'D:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\'
dotenv_path = join(path, '.env')

load_dotenv(dotenv_path)

CR_Token = os.getenv(f'CR_Token')
adm_token = os.getenv(f'ADM_TOKEN')
ASTRA_DB_SECURE_BUNDLE_PATH = os.getenv(f'ASTRA_DB_SECURE_BUNDLE_PATH')
DB_ID = os.getenv(f'DB_ID')

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# with open(ASTRA_TOKEN_PATH, "r") as f:
#     creds = json.load(f)
    # ASTRA_DB_APPLICATION_TOKEN = creds["token"]

ASTRA_DB_APPLICATION_TOKEN = adm_token
try: 
    cluster = Cluster(
        cloud={
            "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
        },
        auth_provider=PlainTextAuthProvider(
            "token",
            ASTRA_DB_APPLICATION_TOKEN,
        ),
    )

    session = cluster.connect()


    session = cluster.connect()
except Exception as e:
    print(e)

In [ ]:
# # This should make a connection to a Cassandra instance your local machine 
# # (127.0.0.1)

# from cassandra.cluster import Cluster
# cluster = Cluster()

# # To establish connection and begin executing queries, need a session
# session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 
# created on portal

#### Set Keyspace

In [14]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project_keyspace')
except Exception as e:
    print(e)

In [15]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [16]:
df.columns # artist, song title, song lenght based on sessionId, itemInSession
            # artist, song "sorted by itemInSession" , 1st and last name based on userId and sessionID
            # 1st and last name based on songListened

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [20]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_query = 'CREATE TABLE IF NOT EXISTS '
query = create_query + "music_library (artist varchar, song varchar, length FLOAT, sessionId INT, itemInSession INT, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)    

In [28]:
from tqdm.auto import tqdm


In [27]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # print((line[0], line[-2], float(line[5]), line[-3], line[3]))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[-2], float(line[5]), int(line[-3]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [41]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = 'SELECT artist, song, length FROM music_library where sessionId = 338 AND itemInSession = 4'
try:
    data = session.execute(query)
except Exception as e:
    print(e)    

In [42]:
for d in data:
    print(d.artist)

Faithless


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [43]:

# Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
#        'level', 'location', 'sessionId', 'song', 'userId'],
#       dtype='object')

create_query = 'CREATE TABLE IF NOT EXISTS '
query = create_query + "music_library_user (artist varchar, song varchar, firstName varchar, lastName varchar, userId INT, sessionId INT, itemInSession INT, PRIMARY KEY (userId, sessionId))"

try:
    session.execute(query)
except Exception as e:
    print(e)    

In [52]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


x = 0

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        if x == 1000:
            break
        x += 1
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_user (artist, song, firstName, lastName, userId, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # print((line[0], line[-2], float(line[5]), line[-3], line[3]))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # print(line[0], line[-2], line[1], line[4], int(line[-1]), int(line[-3]), int(line[3]))
        session.execute(query, (line[0], line[-2], line[1], line[4], int(line[-1]), int(line[-3]), int(line[3])))



0it [00:00, ?it/s]

In [56]:
query = 'SELECT * FROM music_library_user where userid = 10 AND sessionid = 182'
try:
    data = session.execute(query)
except Exception as e:
    print(e)   
for d in data:
    print(d)

Row(userid=10, sessionid=182, artist='Lonnie Gordon', firstname='Sylvie', iteminsession=3, lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [57]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_query = 'CREATE TABLE IF NOT EXISTS '
query = create_query + "music_library_user_name (firstName varchar, lastName varchar, song varchar, PRIMARY KEY (song, firstName, lastName))"

try:
    session.execute(query)
except Exception as e:
    print(e)    

x = 0

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        if x == 1000:
            break
        x += 1
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_user_name (firstName, lastName, song)"
        query = query + " VALUES (%s, %s, %s)"
        # print((line[0], line[-2], float(line[5]), line[-3], line[3]))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1], line[4], line[-2]))

                    

0it [00:00, ?it/s]

In [58]:


query = "SELECT * FROM music_library_user_name where song = 'All Hands Against His Own'"
try:
    data = session.execute(query)
except Exception as e:
    print(e)   
for d in data:
    print(d)

Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [59]:
query = "DROP TABLE IF EXISTS music_library_user_name"
try:
    data = session.execute(query)
except Exception as e:
    print(e)   
for d in data:
    print(d)

query = "DROP TABLE IF EXISTS music_library_user"
try:
    data = session.execute(query)
except Exception as e:
    print(e)   
for d in data:
    print(d)

query = "DROP TABLE IF EXISTS music_library"
try:
    data = session.execute(query)
except Exception as e:
    print(e)   
for d in data:
    print(d)

### Close the session and cluster connection¶

In [60]:
session.shutdown()
cluster.shutdown()